In [3]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC
import face_recognition
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

RuntimeError: Error while calling cudaGetDevice(&the_device_id) in file /tmp/pip-install-5prnabg4/dlib_622639a74c8e49a8a6f0f43c1946fb7e/dlib/cuda/gpu_data.cpp:204. code: 999, reason: unknown error

In [ ]:

# Caminho das pastas
TRAINING_DIR = "./faces"
TRAINING_DIR = os.path.expanduser(TRAINING_DIR)

TRAINING_DIR    

In [13]:

# Função para carregar imagens e labels
def load_images_and_labels(base_dir):
    images = []
    labels = []
    for label in os.listdir(base_dir):
        label_path = os.path.join(base_dir, label)
        if os.path.isdir(label_path):
            for image_file in os.listdir(label_path):
                image_path = os.path.join(label_path, image_file)
                image = face_recognition.load_image_file(image_path)
                images.append(image)
                labels.append(label)
    return images, labels

# Função para realizar data augmentation
def augment_image(image):
    augmented_images = []
    # Conversão para BGR (para uso com OpenCV)
    image_bgr = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    rows, cols, _ = image_bgr.shape

    # Rotação
    for angle in [10, -10]:
        M = cv2.getRotationMatrix2D((cols // 2, rows // 2), angle, 1)
        rotated = cv2.warpAffine(image_bgr, M, (cols, rows))
        augmented_images.append(cv2.cvtColor(rotated, cv2.COLOR_BGR2RGB))

    # Flip horizontal
    flipped = cv2.flip(image_bgr, 1)
    augmented_images.append(cv2.cvtColor(flipped, cv2.COLOR_BGR2RGB))

    return augmented_images

# Função para extrair embeddings faciais
def extract_embeddings(images):
    embeddings = []
    for image in images:
        locations = face_recognition.face_locations(image)
        if locations:
            encoding = face_recognition.face_encodings(image, known_face_locations=locations)[0]
            embeddings.append(encoding)
    return embeddings


In [14]:

# Carregar imagens e labels
images, labels = load_images_and_labels(TRAINING_DIR)

# Data augmentation
augmented_images, augmented_labels = [], []
for img, label in zip(images, labels):
    augmented_images.append(img)
    augmented_labels.append(label)
    for aug_img in augment_image(img):
        augmented_images.append(aug_img)
        augmented_labels.append(label)

# Gerar embeddings
print("Gerando embeddings...")
embeddings = extract_embeddings(augmented_images)

# Codificar labels
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(augmented_labels)


Gerando embeddings...


In [17]:

# Treinamento
X_train, X_test, y_train, y_test = train_test_split(embeddings, encoded_labels, test_size=0.2, random_state=42)

print("Treinando modelo...")
clf = SVC(kernel='linear', probability=True)
clf.fit(X_train, y_train)

# Avaliar modelo
y_pred = clf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Acurácia no conjunto de teste: {accuracy:.2f}")


ValueError: Found input variables with inconsistent numbers of samples: [97, 180]

In [ ]:

# Função para reconhecimento facial
def recognize_faces(test_image_path):
    test_image = face_recognition.load_image_file(test_image_path)
    face_locations = face_recognition.face_locations(test_image)
    face_encodings = face_recognition.face_encodings(test_image, face_locations)

    plt.imshow(test_image)
    plt.axis("off")

    for (top, right, bottom, left), face_encoding in zip(face_locations, face_encodings):
        probabilities = clf.predict_proba([face_encoding])[0]
        best_match_idx = np.argmax(probabilities)
        if probabilities[best_match_idx] > 0.6:  # Threshold de confiança
            name = label_encoder.inverse_transform([best_match_idx])[0]
        else:
            name = "Não reconhecido"

        # Desenhar bounding box e nome
        cv2.rectangle(test_image, (left, top), (right, bottom), (0, 255, 0), 2)
        cv2.putText(test_image, name, (left, top - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

    plt.imshow(cv2.cvtColor(test_image, cv2.COLOR_RGB2BGR))
    plt.show()


In [ ]:

# Testar o algoritmo
test_image_path = "faces-teste/Thiago/20241113_212136.jpg"
recognize_faces(test_image_path)
